In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
quote_page="https://www.zeit.de/wirtschaft/2020-05/solidaritaetszuschlag-steuersenkung-corona-krise-kaufkraft-cdu-csu"

In [4]:
page=requests.get(quote_page)

In [5]:
page.status_code

200

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
print(soup.prettify())

rnav">
     <a class="footer-brand" data-ct-label="logo" data-ct-row="logo" href="https://www.zeit.de/index" title="Nachrichten auf ZEIT ONLINE">
      <svg aria-label="ZEIT ONLINE" class="svg-symbol footer-brand__logo" preserveaspectratio="xMinYMin meet" role="img">
       <title>
        ZEIT ONLINE
       </title>
       <use xlink:href="#svg-logo-zon-black">
       </use>
      </svg>
     </a>
     <nav class="breadcrumbs" data-ct-row="breadcrumbs">
      <h3 class="visually-hidden">
       Navigationspfad
      </h3>
      <ul class="breadcrumbs__list" data-ct-column="1">
       <li class="breadcrumbs__item">
        <a class="breadcrumbs__link" href="https://www.zeit.de/index" title="ZEIT ONLINE">
         Start
        </a>
       </li>
       <li class="breadcrumbs__item">
        <a class="breadcrumbs__link" href="https://www.zeit.de/wirtschaft/index">
         Wirtschaft
        </a>
       </li>
       <li class="breadcrumbs__item">
        Solidaritätszuschlag: Union will 

In [37]:
soup.find_all("p",class_="paragraph article__item")

[<p class="paragraph article__item"><br/>Der <a class="" href="https://www.zeit.de/thema/solidaritaetszuschlag" target="_blank">Solidaritätszuschlag</a> (<span class="rtr-schema-org" itemscope="" itemtype="http://schema.org/Thing"><a class="rtr-entity" href="/thema/solidaritaetszuschlag">Soli</a></span>) könnte bereits zum 1. Juli abgeschafft werden. Das Soli-Ende solle auf diesen Termin vorgezogen werden und vollständig gelten, heißt es in einem Entwurf der AG Wirtschaft der Bundestagsfraktion von CDU und CSU. <a class="" href="https://www.zeit.de/wirtschaft/2019-11/solidaritaetszuschlag-abschaffung-steuerabgaben-einkommen-finanzpolitik" target="_blank">Im vergangenen November</a> hatte der Bundestag beschlossen, dass der Solidaritätszuschlag von 5,5 Prozent in der Einkommens- und Körperschaftssteuer ab Anfang 2021 für 90 Prozent der Steuerzahler wegfallen soll. <a class="" href="https://www.zeit.de/politik/deutschland/2020-02/olaf-scholz-fordert-aus-solidaritaetszuschlag-investitione

In [38]:
type(soup.find_all("p",class_="paragraph article__item"))

bs4.element.ResultSet

In [39]:
type(soup.find_all("p",class_="paragraph article__item")[0])

bs4.element.Tag

In [40]:
type(soup.find_all("p",class_="paragraph article__item")[0].text)

str

In [41]:
soup.find_all("p",class_="paragraph article__item")[0].text

'Der Solidaritätszuschlag (Soli) könnte bereits zum 1. Juli abgeschafft werden. Das Soli-Ende solle auf diesen Termin vorgezogen werden und vollständig gelten, heißt es in einem Entwurf der AG Wirtschaft der Bundestagsfraktion von CDU und CSU. Im vergangenen November hatte der Bundestag beschlossen, dass der Solidaritätszuschlag von 5,5 Prozent in der Einkommens- und Körperschaftssteuer ab Anfang 2021 für 90 Prozent der Steuerzahler wegfallen soll. Die SPD hatte bereits Ende Januar vorgeschlagen, dieses Datum auf den 1. Juli vorzuziehen. \n'

In [42]:
page_address="https://www.immoregion.fr/vente/maison/metz/id-6708871.html"

In [44]:
page=requests.get(page_address)

In [45]:
page.status_code

200

In [46]:
soup = BeautifulSoup(page.content, 'html.parser')

In [48]:
print(soup.prettify())

lse,"has_virtual_visit":false,"has_video":false,"photos_count":11},"items":[{"cover":true,"type":"photo","uri":"\u002Fca\u002F33\u002F66\u002Fe48b2a1f4ba4615514e332d2c1c38595d4d8683f.jpg","order":0},{"cover":false,"type":"photo","uri":"\u002Fca\u002F33\u002F66\u002F3ea56262890e3bbef575d1c9a885993cebc4aad0.jpg","order":1},{"cover":false,"type":"photo","uri":"\u002Fca\u002F33\u002F66\u002Fae7fcef3b2dbb6c9a16cc1739180ddd072edab83.jpg","order":2},{"cover":false,"type":"photo","uri":"\u002Fca\u002F33\u002F66\u002Fbe1375ed1fffe0c0ec65b7de206aa5b7a9b7acb8.jpg","order":3},{"cover":false,"type":"photo","uri":"\u002Fca\u002F33\u002F66\u002Fe92847471a8a14aa7d1104efdd1296be73ce1a99.jpg","order":4},{"cover":false,"type":"photo","uri":"\u002Fca\u002F33\u002F66\u002F0f25ad3b1c8a903d37a447748887147f0891b6b5.jpg","order":5},{"cover":false,"type":"photo","uri":"\u002Fca\u002F33\u002F66\u002F7eef09b358fb2264915ca2fbf3f043a2408bae9c.jpg","order":6},{"cover":false,"type":"photo","uri":"\u002Fca\u002F33\u00

In [55]:
soup.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")

[<div class="feature-bloc-content-specification-content-response"><div>310 000 €</div></div>,
 <div class="feature-bloc-content-specification-content-response"><div>NC</div></div>,
 <div class="feature-bloc-content-specification-content-response"><div>Non</div></div>,
 <div class="feature-bloc-content-specification-content-response"><div>88,65 m²</div></div>,
 <div class="feature-bloc-content-specification-content-response"><div>4,38 ares</div></div>,
 <div class="feature-bloc-content-specification-content-response"><div>2</div></div>,
 <div class="feature-bloc-content-specification-content-response"><div>2017</div></div>,
 <div class="feature-bloc-content-specification-content-response"><div class="feature-bloc-content-specification-content-response-energy-performance"><span class="energy-number"></span><label class="modal-label modal-bg modal-energy-label modal-energy" for="modal-energy"><span class="energy fr-dpe-nc">NC</span></label></div></div>,
 <div class="feature-bloc-content-s